# I have worked on both Data related to Toronto and New york 

# New York Data

In [ ]:
import numpy as np
import pandas as pd 
import json
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import folium # map rendering library
!conda install -c conda-forge beautifulsoup4 --yes
!pip install lxml html5lib beautifulsoup4
from bs4 import BeautifulSoup
import lxml.html as lh
print('Libraries imported.')

In [2]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

In [3]:
ny_data = newyork_data['features']
ny_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [4]:
column_names = ['City''Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
newy_data = pd.DataFrame(columns=column_names)
for data in ny_data:
    borough = ny_name = data['properties']['borough'] 
    ny_name = data['properties']['name']
        
    ny_latlon = data['geometry']['coordinates']
    ny_lat =ny_latlon[1]
    ny_lon = ny_latlon[0]
    
    newy_data =newy_data.append({'Borough': borough,
                                          'Neighborhood': ny_name,
                                          'Latitude': ny_lat,
                                          'Longitude': ny_lon}, ignore_index=True)

In [5]:
newy_data['City']='New York'

In [6]:
newy_data.drop(['CityBorough'],axis=1,inplace=True)
newy_data.head()

,Neighborhood,Latitude,Longitude,Borough,City
0,Wakefield,40.894705,-73.847201,Bronx,New York
1,Co-op City,40.874294,-73.829939,Bronx,New York
2,Eastchester,40.887556,-73.827806,Bronx,New York
3,Fieldston,40.895437,-73.905643,Bronx,New York
4,Riverdale,40.890834,-73.912585,Bronx,New York


In [7]:

address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address,timeout=10000)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


In [8]:
def citymap(cityname,countryname,dataframe):
    # create map
    address = cityname + ',' + countryname

    geolocator = Nominatim(user_agent="my-application")
    location = geolocator.geocode(address,timeout=10000)
    latitude = location.latitude
    longitude = location.longitude
    my_map = folium.Map(location=[latitude, longitude], zoom_start=10)

    # set color scheme for the Borough
    borough_name = dataframe['Borough'].unique().tolist()
    colnum = dataframe['Borough'].unique().size
    x = np.arange(colnum)
    ys = [i+x+(i*x)**2 for i in range(colnum)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map

    for lat, lon, neighborhood, borough in zip(dataframe['Latitude'], dataframe['Longitude'], dataframe['Neighborhood'], dataframe['Borough']):
        cluster = borough_name.index(borough)
        label = '{}, {}'.format(neighborhood, borough)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster],
            fill=True,
            fill_color=rainbow[cluster],
            fill_opacity=0.7).add_to(my_map)
       
    
    return my_map

In [9]:
print('The Data for New york has {} boroughs and {} neighborhoods.'.format(
        len(newy_data['Borough'].unique()),
        newy_data.shape[0]
    )
)


The Data for New york has 5 boroughs and 306 neighborhoods.


In [10]:
print("The Borough's in Newyork are:",newy_data['Borough'].unique())

The Borough's in Newyork are: ['Bronx' 'Manhattan' 'Brooklyn' 'Queens' 'Staten Island']


In [205]:
citymap('New York','USA',newy_data)

# Toronto Data

In [12]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wikipage = requests.get(url)
wikipage.text[:100]

#Create a new pd DataFrame
toronto = pd.DataFrame()

#use beautifulsoup to read the wikipage
soup = BeautifulSoup(wikipage.text, 'lxml')
wikitable = soup.find_all('table')[0] 

row_marker = 0

for row in wikitable.find_all('tr'):
    column_marker = 0
    columns = row.find_all('td')
    for column in columns:
        toronto.loc[row_marker,column_marker] = column.get_text()
        column_marker += 1
    row_marker += 1

#rename column names
toronto.rename(columns={0:'PostalCode',1:'Borough',2:'Neighborhood'}, inplace=True)
#drop all rows with unassigned borough
toronto = toronto[toronto.Borough!='Not assigned']
toronto.reset_index(drop=True,inplace=True)

#create a new dataframe toronto_neighbor to find all neighborhoods with the location information
trn_Data= pd.DataFrame(columns = ['City','Borough','Neighborhood'])

for ii in range(toronto.shape[0]):
    borough = toronto.loc[ii,'Borough']
    postalcode = toronto.loc[ii,'PostalCode']
    neighborhood = toronto.loc[ii,'Neighborhood'][:-1] #delete the last character \n
    
    #if the neighborhood name is not assigned, than the neighorhood name is same as borough
    if neighborhood == 'Not assigned':
        neighborhood = borough
    
    #find the location data, ignore the neighborhoods that are unable to be located by Nominatim
    geolocator = Nominatim(user_agent="mycapstoneproject")
    location = geolocator.geocode("{},{},Toronto,Ontario,Canada".format(neighborhood,borough))
    
    #try one more searching
    if location is None: 
        location = geolocator.geocode("{},Toronto,Ontario,Canada".format(neighborhood))
        
    if location is None: 
        print('',end='')
    else:
        trn_Data = trn_Data.append({'City': 'Toronto',
                                                'Borough': borough,
                                                'Neighborhood': neighborhood,
                                                'Latitude': location.latitude,
                                                'Longitude': location.longitude
                                               }, ignore_index=True)

trn_Data[['Borough','waste']] = trn_Data.Borough.str.split("\n",expand=True)
trn_Data.drop(['waste'],axis=1,inplace=True)
trn_Data.head()

,City,Borough,Neighborhood,Latitude,Longitude
0,Toronto,North York,Parkwoods,43.758800,-79.320197
1,Toronto,North York,Victoria Village,43.732658,-79.311189
2,Toronto,Downtown Toronto,"Regent Park, Harbourfront",43.640769,-79.379892
3,Toronto,North York,"Lawrence Manor, Lawrence Heights",43.715283,-79.443914
4,Toronto,Etobicoke,"Islington Avenue, Humber Valley Village",43.668499,-79.533970


In [14]:
#trn_data=pd.read_csv('Toronto_data.csv')
#trn_data.drop(['Unnamed: 0'],axis=1,inplace=True)
trn_Data.to_csv('Toronto_data.csv')

In [17]:
print('The Data for Toronto has {} boroughs and {} neighborhoods.'.format(
        len(trn_Data['Borough'].unique()),
        trn_Data.shape[0]
    )
)
print(trn_Data['Borough'].unique())

The Data for Toronto has 9 boroughs and 54 neighborhoods.
['North York' 'Downtown Toronto' 'Etobicoke' 'Scarborough' 'East York'
 'East Toronto' 'West Toronto' 'Central Toronto' 'York']


In [206]:
citymap('Toronto','Canada',trn_Data)

In [49]:
CLIENT_ID = 'RZKKI01CSCDD2RI5UTGWFK4JHT3Z40KG15HGYGQH0LQB3PVG' # your Foursquare ID
CLIENT_SECRET = 'C1JMQPQNCXXQJCIEUFJMSM0QZPGMPNJQQGLIWYE3KBZSNPKA' # your Foursquare Secret
VERSION = '20200610' # Foursquare API version
LIMIT=20000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: RZKKI01CSCDD2RI5UTGWFK4JHT3Z40KG15HGYGQH0LQB3PVG
CLIENT_SECRET:C1JMQPQNCXXQJCIEUFJMSM0QZPGMPNJQQGLIWYE3KBZSNPKA


In [50]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'],   
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue',  
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
#newy_venues=getNearbyVenues(names=newy_data['Neighborhood'],
                                   latitudes=newy_data['Latitude'],
                                   longitudes=newy_data['Longitude']
                                  )

Wakefield
Co-op City
Eastchester
Fieldston
Riverdale
Kingsbridge
Marble Hill
Woodlawn
Norwood
Williamsbridge
Baychester
Pelham Parkway
City Island
Bedford Park
University Heights
Morris Heights
Fordham
East Tremont
West Farms
High  Bridge
Melrose
Mott Haven
Port Morris
Longwood
Hunts Point
Morrisania
Soundview
Clason Point
Throgs Neck
Country Club
Parkchester
Westchester Square
Van Nest
Morris Park
Belmont
Spuyten Duyvil
North Riverdale
Pelham Bay
Schuylerville
Edgewater Park
Castle Hill
Olinville
Pelham Gardens
Concourse
Unionport
Edenwald
Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker

In [52]:
newy_venues.to_csv('NewYorkVenues.csv')

In [55]:
#trn_venues=getNearbyVenues(names=trn_Data['Neighborhood'],
                                   latitudes=trn_Data['Latitude'],
                                   longitudes=trn_Data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Glencairn
Don Mills
Woodbine Heights
St. James Town
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Thorncliffe Park
Dufferin, Dovercourt Village
Scarborough Village
Bayview Village
Downsview
The Danforth West, Riverdale
York Mills, Silver Hills
Downsview
Commerce Court, Victoria Hotel
Humber Summit
Downsview
Studio District
Humberlea, Emery
Willowdale, Willowdale East
Downsview
Lawrence Park
Roselawn
Runnymede, The Junction North
Weston
York Mills West
Davisville North
Westmount
Willowdale, Willowdale West
Parkdale, Roncesvalles
Agincourt
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Steeles West, L'Amoreaux West
Rosedale
Alderwood, Long Branch
Upper Rouge
St. James Town, Cabbagetown
Church and Wellesley


In [57]:

trn_venues.to_csv('Toronto_venues.csv')

In [108]:
all_data=pd.concat([newy_data,trn_Data])
all_data = all_data.rename(columns={'Neighborhood': 'NeighborhoodName'})

In [59]:
newy_venues['city']='New York'
trn_venues['city']='Toronto'

In [113]:
all_venues= pd.concat([newy_venues,trn_venues])
all_venues.shape
all_venues = all_venues.rename(columns={'Neighborhood': 'NeighborhoodName'})

In [114]:
all_venues.head()

,NeighborhoodName,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Category,city
0,Wakefield,40.894705,-73.847201,Lollipops Gelato,Dessert Shop,New York
1,Wakefield,40.894705,-73.847201,Carvel Ice Cream,Ice Cream Shop,New York
2,Wakefield,40.894705,-73.847201,Walgreens,Pharmacy,New York
3,Wakefield,40.894705,-73.847201,Rite Aid,Pharmacy,New York
4,Wakefield,40.894705,-73.847201,Dunkin',Donut Shop,New York


In [115]:
allvenues_onehot = pd.get_dummies(all_venues, columns = ['Venue Category'], prefix="", prefix_sep="")
allvenues_onehot = allvenues_onehot.drop('Venue',axis = 1)

In [116]:
allvenues_grouped = allvenues_onehot.groupby('NeighborhoodName',axis = 0).sum().reset_index()
allvenues_grouped = all_data.join(allvenues_grouped.set_index('NeighborhoodName'), on='NeighborhoodName')

In [170]:
allvenues_grouped.dropna(inplace=True)
allvenues_grouped.drop(['Neighborhood Latitude'],1,inplace=True)
allvenues_grouped.drop(['Neighborhood Longitude'],1,inplace=True)
allvenues_grouped.head()

,NeighborhoodName,Latitude,Longitude,Borough,City,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,Airport Terminal,...,Warehouse Store,Waste Facility,Waterfront,Weight Loss Center,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Wakefield,40.894705,-73.847201,Bronx,New York,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Co-op City,40.874294,-73.829939,Bronx,New York,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Eastchester,40.887556,-73.827806,Bronx,New York,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Fieldston,40.895437,-73.905643,Bronx,New York,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Riverdale,40.890834,-73.912585,Bronx,New York,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [173]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

columns = ['City','Borough','NeighborhoodName','Latitude','Longitude','Total Number of Venues']
indicators = ['st', 'nd', 'rd']
# create columns according to number of top venues

for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
allvenues_sorted = pd.DataFrame(columns = columns)
#allvenues_sorted = allvenues_grouped[['City','Borough','NeighborhoodName']]

for ind in range(allvenues_grouped.shape[0]):
    allvenues_sorted.loc[ind, 'City'] = allvenues_grouped.iloc[ind].City
    allvenues_sorted.loc[ind, 'Borough'] = allvenues_grouped.iloc[ind].Borough
    allvenues_sorted.loc[ind, 'NeighborhoodName'] = allvenues_grouped.iloc[ind].NeighborhoodName
    allvenues_sorted.loc[ind, 'Latitude'] = allvenues_grouped.iloc[ind].Latitude
    allvenues_sorted.loc[ind, 'Longitude'] = allvenues_grouped.iloc[ind].Longitude
    #allvenues_sorted.loc[ind, 'Total Number of Venues'] = allvenues_grouped.iloc[ind,5:].sum()
    allvenues_sorted.iloc[ind, 6:] = return_most_common_venues(allvenues_grouped.iloc[ind, 5:], num_top_venues)

allvenues_sorted.head()

,City,Borough,NeighborhoodName,Latitude,Longitude,Total Number of Venues,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Bronx,Wakefield,40.8947,-73.8472,NaN,Pharmacy,Food,Dessert Shop,Donut Shop,Laundromat,Ice Cream Shop,Pizza Place,Sandwich Place,Gas Station,Factory
1,New York,Bronx,Co-op City,40.8743,-73.8299,NaN,Bus Station,Basketball Court,Pizza Place,Fast Food Restaurant,Grocery Store,Chinese Restaurant,Park,Baseball Field,Pharmacy,Bagel Shop
2,New York,Bronx,Eastchester,40.8876,-73.8278,NaN,Caribbean Restaurant,Deli / Bodega,Bus Station,Diner,Food & Drink Shop,Automotive Shop,Bakery,Donut Shop,Bowling Alley,Pizza Place
3,New York,Bronx,Fieldston,40.8954,-73.9056,NaN,Bus Station,River,Plaza,Yoga Studio,Fish & Chips Shop,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant
4,New York,Bronx,Riverdale,40.8908,-73.9126,NaN,Bus Station,Park,Medical Supply Store,Gym,Baseball Field,Food Truck,Bank,Plaza,Farm,Fast Food Restaurant


In [174]:
allvenues_sorted.drop(['Total Number of Venues'],axis=1,inplace=True)
allvenues_sorted.head()

,City,Borough,NeighborhoodName,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,Bronx,Wakefield,40.8947,-73.8472,Pharmacy,Food,Dessert Shop,Donut Shop,Laundromat,Ice Cream Shop,Pizza Place,Sandwich Place,Gas Station,Factory
1,New York,Bronx,Co-op City,40.8743,-73.8299,Bus Station,Basketball Court,Pizza Place,Fast Food Restaurant,Grocery Store,Chinese Restaurant,Park,Baseball Field,Pharmacy,Bagel Shop
2,New York,Bronx,Eastchester,40.8876,-73.8278,Caribbean Restaurant,Deli / Bodega,Bus Station,Diner,Food & Drink Shop,Automotive Shop,Bakery,Donut Shop,Bowling Alley,Pizza Place
3,New York,Bronx,Fieldston,40.8954,-73.9056,Bus Station,River,Plaza,Yoga Studio,Fish & Chips Shop,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant
4,New York,Bronx,Riverdale,40.8908,-73.9126,Bus Station,Park,Medical Supply Store,Gym,Baseball Field,Food Truck,Bank,Plaza,Farm,Fast Food Restaurant


In [177]:
from sklearn.cluster import KMeans
kclusters = 6
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(all_venues_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 
allvenues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [157]:
allvenues_sorted.drop(['Cluster Labels'],axis=1,inplace=True)

In [ ]:
allvenues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

all_merged = all_data


all_merged = all_merged.join(all_venues_sorted.set_index('NeighborhoodName'), on='NeighborhoodName')

all_merged.head() # check the last columns!

In [178]:
allvenues_sorted.head()


,Cluster Labels,City,Borough,NeighborhoodName,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2,New York,Bronx,Wakefield,40.8947,-73.8472,Pharmacy,Food,Dessert Shop,Donut Shop,Laundromat,Ice Cream Shop,Pizza Place,Sandwich Place,Gas Station,Factory
1,2,New York,Bronx,Co-op City,40.8743,-73.8299,Bus Station,Basketball Court,Pizza Place,Fast Food Restaurant,Grocery Store,Chinese Restaurant,Park,Baseball Field,Pharmacy,Bagel Shop
2,2,New York,Bronx,Eastchester,40.8876,-73.8278,Caribbean Restaurant,Deli / Bodega,Bus Station,Diner,Food & Drink Shop,Automotive Shop,Bakery,Donut Shop,Bowling Alley,Pizza Place
3,2,New York,Bronx,Fieldston,40.8954,-73.9056,Bus Station,River,Plaza,Yoga Studio,Fish & Chips Shop,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant
4,2,New York,Bronx,Riverdale,40.8908,-73.9126,Bus Station,Park,Medical Supply Store,Gym,Baseball Field,Food Truck,Bank,Plaza,Farm,Fast Food Restaurant


In [207]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=15)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(allvenues_sorted['Latitude'], allvenues_sorted['Longitude'], allvenues_sorted['NeighborhoodName'], allvenues_sorted['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [208]:
for ii in range(6):
    num = allvenues_sorted[allvenues_sorted['Cluster Labels'] == ii].shape[0]
    print('Total number of neighborhoods in cluster {} is {}'.format(ii, num))

Total number of neighborhoods in cluster 0 is 46
Total number of neighborhoods in cluster 1 is 2
Total number of neighborhoods in cluster 2 is 184
Total number of neighborhoods in cluster 3 is 105
Total number of neighborhoods in cluster 4 is 17
Total number of neighborhoods in cluster 5 is 5


In [191]:
def clustermap(cityname,countryname,dataframe):
    # create map
    address = cityname + ',' + countryname

    geolocator = Nominatim(user_agent="my-application")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    my_map = folium.Map(location=[latitude, longitude], zoom_start=10)

    # set color scheme for the Cluster_Labels
    colnum = 9
    x = np.arange(colnum)
    ys = [i+x+(i*x)**2 for i in range(colnum)]
    colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
    rainbow = [colors.rgb2hex(i) for i in colors_array]

    # add markers to the map

    for lat, lon, neighborhood, borough, cluster_labels in zip(dataframe['Latitude'], 
                                                               dataframe['Longitude'], 
                                                               dataframe['NeighborhoodName'], 
                                                               dataframe['Borough'],
                                                               dataframe['Cluster Labels']):
        cluster = cluster_labels 
        label = '{}, {},Cluster:{}'.format(neighborhood, borough,cluster_labels)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lon],
            radius=5,
            popup=label,
            color=rainbow[cluster],
            fill=True,
            fill_color=rainbow[cluster],
            fill_opacity=0.7).add_to(my_map)
       
    
    return my_map

# Cluster:0

In [189]:
allvenues_sorted.loc[allvenues_sorted['Cluster Labels'] == 0, allvenues_sorted.columns[[1] + list(range(5, allvenues_sorted.shape[1]))]]

,City,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
34,New York,-73.8885,Italian Restaurant,Deli / Bodega,Pizza Place,Bakery,Dessert Shop,Donut Shop,Bank,Bar,Fish Market,Food & Drink Shop
46,New York,-74.0306,Spa,Italian Restaurant,Pizza Place,Greek Restaurant,Bagel Shop,Grocery Store,Bar,American Restaurant,Sandwich Place,Hookah Bar
49,New York,-73.9542,Bar,Pizza Place,Cocktail Bar,Coffee Shop,French Restaurant,Sushi Restaurant,Café,Deli / Bodega,Yoga Studio,Grocery Store
59,New York,-73.9649,Bar,Mexican Restaurant,Wine Shop,Gourmet Shop,Cocktail Bar,Café,Thai Restaurant,Yoga Studio,Bakery,Japanese Restaurant
62,New York,-73.9253,Bar,Mexican Restaurant,Coffee Shop,Deli / Bodega,Pizza Place,Thrift / Vintage Store,Discount Store,Bakery,Pharmacy,Café
64,New York,-73.9938,Yoga Studio,Deli / Bodega,Park,Italian Restaurant,Pizza Place,Gym,Bakery,Mexican Restaurant,American Restaurant,Juice Bar
65,New York,-73.9986,Bar,Pizza Place,Playground,Coffee Shop,Yoga Studio,Ice Cream Shop,Deli / Bodega,Italian Restaurant,Cocktail Bar,Bakery
66,New York,-73.9947,Italian Restaurant,Coffee Shop,Pizza Place,Cocktail Bar,Bakery,Bar,Wine Shop,Spa,Ice Cream Shop,Food & Drink Shop
68,New York,-73.9944,Bar,Furniture / Home Store,Italian Restaurant,Art Gallery,Pizza Place,Deli / Bodega,Mexican Restaurant,Chinese Restaurant,Coffee Shop,Gym / Fitness Center
69,New York,-73.9729,Italian Restaurant,Wine Shop,Pizza Place,Flower Shop,Playground,Theater,Wine Bar,French Restaurant,Coffee Shop,Cocktail Bar


In [192]:

cluster0 = allvenues_sorted[allvenues_sorted['Cluster Labels'] == 0]
clustermap('New York','USA',cluster0)

# Clustr :1

In [193]:
allvenues_sorted.loc[allvenues_sorted['Cluster Labels'] == 1, allvenues_sorted.columns[[1] + list(range(5, allvenues_sorted.shape[1]))]]

,City,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
311,Toronto,-79.3459,Clothing Store,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Restaurant,Bank,Bakery,Sandwich Place,Women's Store,Convenience Store
313,Toronto,-79.3459,Clothing Store,Coffee Shop,Japanese Restaurant,Fast Food Restaurant,Restaurant,Bank,Bakery,Sandwich Place,Women's Store,Convenience Store


In [194]:
cluster1 = allvenues_sorted[allvenues_sorted['Cluster Labels'] == 1]
clustermap('Toronto','Canada',cluster1)

# Cluster:2

In [195]:
allvenues_sorted.loc[allvenues_sorted['Cluster Labels'] == 2, allvenues_sorted.columns[[1] + list(range(5, allvenues_sorted.shape[1]))]]

,City,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,New York,-73.8472,Pharmacy,Food,Dessert Shop,Donut Shop,Laundromat,Ice Cream Shop,Pizza Place,Sandwich Place,Gas Station,Factory
1,New York,-73.8299,Bus Station,Basketball Court,Pizza Place,Fast Food Restaurant,Grocery Store,Chinese Restaurant,Park,Baseball Field,Pharmacy,Bagel Shop
2,New York,-73.8278,Caribbean Restaurant,Deli / Bodega,Bus Station,Diner,Food & Drink Shop,Automotive Shop,Bakery,Donut Shop,Bowling Alley,Pizza Place
3,New York,-73.9056,Bus Station,River,Plaza,Yoga Studio,Fish & Chips Shop,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant
4,New York,-73.9126,Bus Station,Park,Medical Supply Store,Gym,Baseball Field,Food Truck,Bank,Plaza,Farm,Fast Food Restaurant
...,...,...,...,...,...,...,...,...,...,...,...,...
349,Toronto,-79.2785,Chinese Restaurant,Bakery,Cantonese Restaurant,Hong Kong Restaurant,Train Station,Vietnamese Restaurant,Korean Restaurant,Asian Restaurant,Shopping Mall,Food Court
353,Toronto,-79.318,Grocery Store,Chinese Restaurant,Fast Food Restaurant,Pizza Place,Electronics Store,Coffee Shop,Breakfast Spot,Bank,Sandwich Place,Pharmacy
355,Toronto,-79.5452,Pizza Place,Skating Rink,Pub,Pharmacy,Coffee Shop,Gym,Sandwich Place,Yoga Studio,Ethiopian Restaurant,Event Service
356,Toronto,-79.1658,Park,Fast Food Restaurant,Yoga Studio,Cultural Center,Event Service,Event Space,Exhibit,Factory,Falafel Restaurant,Farm


In [196]:
cluster2 = allvenues_sorted[allvenues_sorted['Cluster Labels'] == 2]
clustermap('New York','USA',cluster2)


In [197]:
clustermap('Toronto','Canada',cluster2)

# Cluster :3

In [198]:
allvenues_sorted.loc[allvenues_sorted['Cluster Labels'] == 3, allvenues_sorted.columns[[1] + list(range(5, allvenues_sorted.shape[1]))]]

,City,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,New York,-73.9028,Pizza Place,Bar,Latin American Restaurant,Sandwich Place,Bakery,Supermarket,Mexican Restaurant,Spanish Restaurant,Fast Food Restaurant,Donut Shop
6,New York,-73.9107,Sandwich Place,Coffee Shop,Gym,Yoga Studio,Video Game Store,Tennis Stadium,Seafood Restaurant,Miscellaneous Shop,Bank,Pharmacy
7,New York,-73.8673,Pub,Deli / Bodega,Pizza Place,Playground,Italian Restaurant,Food & Drink Shop,Grocery Store,Park,Bar,Bakery
8,New York,-73.8794,Pizza Place,Bank,Park,Pharmacy,Food Truck,Bus Station,Pet Store,Caribbean Restaurant,Sandwich Place,Mobile Phone Shop
11,New York,-73.8548,Bus Station,Italian Restaurant,Frozen Yogurt Shop,Pizza Place,Mexican Restaurant,Bank,Sandwich Place,Coffee Shop,Gym / Fitness Center,Metro Station
...,...,...,...,...,...,...,...,...,...,...,...,...
348,Toronto,-79.4397,Tibetan Restaurant,Café,Italian Restaurant,Pharmacy,Indian Restaurant,Restaurant,Diner,Pizza Place,Bakery,Tattoo Parlor
350,Toronto,-79.3973,Italian Restaurant,Sushi Restaurant,Coffee Shop,Trail,Park,Convenience Store,Pub,Bank,Bar,Café
351,Toronto,-79.3987,Café,Coffee Shop,Restaurant,Italian Restaurant,Japanese Restaurant,Gym,Bookstore,Bakery,Yoga Studio,Music School
352,Toronto,-79.4759,Café,Coffee Shop,Bakery,Bank,Pizza Place,Bookstore,Gastropub,Liquor Store,Falafel Restaurant,Boutique


In [199]:
cluster3 = allvenues_sorted[allvenues_sorted['Cluster Labels'] == 3]
clustermap('New York','USA',cluster3)

In [200]:
clustermap('Toronto','Canada',cluster3)

# Cluster:4

In [201]:
allvenues_sorted.loc[allvenues_sorted['Cluster Labels'] == 4, allvenues_sorted.columns[[1] + list(range(5, allvenues_sorted.shape[1]))]]

,City,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
96,New York,-73.9588,Coffee Shop,Pizza Place,Wine Bar,Yoga Studio,American Restaurant,Bakery,Jewelry Store,Vegetarian / Vegan Restaurant,Bar,Juice Bar
97,New York,-73.958,Coffee Shop,Bar,American Restaurant,Pizza Place,Yoga Studio,Breakfast Spot,Wine Bar,Japanese Restaurant,Mexican Restaurant,Cycle Studio
114,New York,-73.9817,Hotel,Coffee Shop,Theater,Bakery,Clothing Store,Sushi Restaurant,Japanese Restaurant,Pizza Place,Cuban Restaurant,Burger Joint
116,New York,-74.0031,Coffee Shop,Art Gallery,Ice Cream Shop,Bakery,Café,American Restaurant,Bar,Italian Restaurant,Cocktail Bar,Seafood Restaurant
128,New York,-74.0107,Coffee Shop,Pizza Place,Hotel,Café,Italian Restaurant,American Restaurant,Event Space,Mexican Restaurant,Cocktail Bar,Sandwich Place
139,New York,-73.9392,Coffee Shop,Hotel,Pizza Place,Bar,Café,Mexican Restaurant,Supermarket,Deli / Bodega,Gym / Fitness Center,Office
169,New York,-73.7353,Coffee Shop,Clothing Store,Sandwich Place,Middle Eastern Restaurant,Café,Bar,Chinese Restaurant,Hotel,Ramen Restaurant,Tanning Salon
244,New York,-74.1896,Coffee Shop,Art Gallery,Ice Cream Shop,Bakery,Café,American Restaurant,Bar,Italian Restaurant,Cocktail Bar,Seafood Restaurant
247,New York,-73.9533,Coffee Shop,Café,Yoga Studio,Bookstore,Gym / Fitness Center,Italian Restaurant,Gym,Wine Shop,Pizza Place,Japanese Restaurant
249,New York,-74.0054,Coffee Shop,Hotel,Cocktail Bar,Park,American Restaurant,Spa,Gym / Fitness Center,French Restaurant,Café,Yoga Studio


In [202]:
cluster4 = allvenues_sorted[allvenues_sorted['Cluster Labels'] == 4]
clustermap('New York','USA',cluster4)

In [203]:
clustermap('Toronto','Canada',cluster4)

# Cluster 5

In [204]:
allvenues_sorted.loc[allvenues_sorted['Cluster Labels'] == 5, allvenues_sorted.columns[[1] + list(range(5, allvenues_sorted.shape[1]))]]

,City,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
115,New York,-73.9783,Korean Restaurant,Hotel,Sandwich Place,Coffee Shop,Pizza Place,Bar,Japanese Restaurant,Gym / Fitness Center,Mediterranean Restaurant,Fried Chicken Joint
180,New York,-73.8128,Korean Restaurant,Hotel,Sandwich Place,Coffee Shop,Pizza Place,Bar,Japanese Restaurant,Gym / Fitness Center,Mediterranean Restaurant,Fried Chicken Joint
250,New York,-73.9887,Korean Restaurant,Hotel,Dessert Shop,American Restaurant,Café,Burger Joint,Coffee Shop,Japanese Restaurant,Hotel Bar,Gym / Fitness Center
338,Toronto,-79.4138,Bubble Tea Shop,Korean Restaurant,Pizza Place,Ramen Restaurant,Dessert Shop,Vietnamese Restaurant,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Greek Restaurant
347,Toronto,-79.4138,Bubble Tea Shop,Korean Restaurant,Pizza Place,Ramen Restaurant,Dessert Shop,Vietnamese Restaurant,Japanese Restaurant,Sushi Restaurant,Coffee Shop,Greek Restaurant


# Deatiled Discussion and Conclusion are given in Report